In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_mse import LstmMse
from models_mle import LstmMle_1
from models_sub_net_ls import LstmMse_LatentSpace
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse
from predictor import PredictorMle

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters

In [3]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/MLE_analysis/artifical_2_signals_errors.csv',
        "droped_feature" : [
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/artifical_2_signalsFold xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "n_hidden_fc" : 75,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/artifical_2_signals.csv",
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from NewBlade Training Data

In [ ]:
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02, 1.45531178e+04, 1.37766733e+03, 6.50149764e-01] 
var_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]

### Mean and Variance from Artifical Training Data

In [ ]:
mean_training_data= [-5.31764899e-02, -3.98576146e-04, -8.82773455e+05,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01,  1.23172374e-02]
var_training_data = [1.28792583e-01, 1.21258617e-03, 2.90245238e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03, 5.01164766e+01]

In [14]:
mean_training_data= [-0.00393712, -0.01294209]
var_training_data = [49.18936568,  0.34270256]

### Mean and Variance from phm Dataset
droped features=["ongoing time", "up time", "RLU", "runnum"] + <br>
['FIXTURESHUTTERPOSITION_0.0','FIXTURESHUTTERPOSITION_1.0', 'FIXTURESHUTTERPOSITION_2.0', 'FIXTURESHUTTERPOSITION_3.0', <br>
'FIXTURESHUTTERPOSITION_255.0', "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", "ETCHAUX2SOURCETIMER"]

In [ ]:
mean_training_data =[ 0.0632522,0.10388593, 0.09563544, 0.0777276, 0.22081628, 0.08311531, 0.01382531,
                     0.09862897, 0.07814727, -0.0185826, 0.1000127, -0.0161782, -0.22541928]
var_training_data =[0.90316232, 0.97237671, 0.98547017, 0.92090347, 1.18086523, 0.92393987,
                    0.41744699, 0.97142703, 0.92604794, 0.68786855, 1.25019607, 0.50023143, 0.69425608]

50.000 Dataset:

In [55]:
mean_training_data =[ 0.21683119,  0.32121513,  0.31925213,  0.20097501,  0.45164471,  0.22914814,
  0.11604865,  0.27421592,  0.24393222, -0.13974937, -0.09739598, -0.07313758,
  0.18198089]
var_training_data =[0.75261122, 0.90482986, 0.91105327, 0.75504036, 1.07026701, 0.76708319,
 0.35172769, 0.83004988, 0.76964675, 0.57386915, 0.45912309, 0.2955709,
 1.61493449]

## Create DataLoader

In [15]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(12000, 3)


In [16]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])
Data of batch: 1
Size of input data: torch.Size([50, 100, 3])
Size of target data: torch.Size([50, 3])


## Define Model and load Parameters of trained model
### Model for MSE 

In [6]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

RuntimeError: Error(s) in loading state_dict for LstmMse:
	Missing key(s) in state_dict: "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "fc_y_hat.weight", "fc_y_hat.bias", "fc_tau.weight", "fc_tau.bias". 

### Model for MLE

In [4]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc=param['model']['n_hidden_fc'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### Model for Latent Space Analysis

In [ ]:
model = LstmMultiTaskLearning(batch_size=param['model']['batch_size'], 
                              input_dim=param['model']['input_size'], 
                              n_hidden_lstm=param['model']['n_hidden_lstm'], 
                              n_layers=param['model']['lstm_layer'],
                              dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                              dropout_rate_fc= param['model']['dropout_rate_fc'],
                              n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                              n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls']      
                              )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

## Initialize Trainer
### Predictor for MSE Model

In [ ]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                         )

### Predictor for MLE Model

In [5]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                         no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                         )

### Predictor for Latent Space Model

In [ ]:
predictor = LstmMse_LatentSpace(model=model,
                                path_data=param["data"]["path"],
                                columns_to_ignore=param["data"]["droped_feature"],
                                threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                                )

## Predict

In [20]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
End of prediction.


## Tag anomalous samples

In [6]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [7]:
results.head()

,ID,sine_signal target,sawtooth_signal target,sine_signal mu predicted,sawtooth_signal mu predicted,sine_signal sigma predicted,sawtooth_signal sigma predicted,normalised residual,sine_signal normalised residual,sawtooth_signal normalised residual,Anomaly Sensor_1,Anomaly Sensor_2
0,100.0,-0.030039,-1.827004,0.151435,0.361860,0.102231,1.325267,-1.713393,-1.775146,-1.651641,0,0
1,101.0,0.182088,-1.640144,0.200949,-1.329780,0.089960,0.268259,-0.683311,-0.209664,-1.156959,0,0
2,102.0,0.185946,-1.844197,0.352622,-1.236555,0.091021,0.220631,-2.292646,-1.831187,-2.754104,0,1
3,103.0,0.553550,-0.993583,0.454444,-1.120401,0.092670,0.214767,0.829971,1.069451,0.590491,0,0
4,104.0,0.596407,-0.941494,0.640806,-0.980928,0.096542,0.202603,-0.132626,-0.459889,0.194637,0,0


## Combine prediction data with data which was not consider for inference

In [8]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [9]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)